In [ ]:
import socket
import os
import struct
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
import time
import random

AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding

# Simulate handshake and key exchange with a simple Diffie-Hellman key exchange
def generate_session_key():
    return get_random_bytes(AES_KEY_SIZE)

def add_random_padding(data):
    padding_length = random.randint(0, RANDOM_PADDING_MAX)
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

def encrypt_data(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data, padding_length = add_random_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    return nonce + struct.pack("B", padding_length) + ciphertext + tag

def decrypt_data(record, key):
    nonce = record[:AES_NONCE_SIZE]
    padding_length = struct.unpack("B", record[AES_NONCE_SIZE:AES_NONCE_SIZE + 1])[0]
    tag_start = -16
    ciphertext = record[AES_NONCE_SIZE + 1:tag_start]
    tag = record[tag_start:]

    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = cipher.decrypt_and_verify(ciphertext, tag)
    
    return padded_data[:-padding_length] if padding_length > 0 else padded_data

def handle_client_connection(client_socket):
    try:
        print("Handshake initiated.")
        
        # Simulate handshake and session key exchange
        client_socket.send(b"HELLO SERVER, PLEASE SEND YOUR ENCRYPTION METHOD")  # Handshake message
        message = client_socket.recv(1024)
        print(f"Received from client: {message.decode()}")

        # Agree on encryption method (e.g., AES) and send back session key
        session_key = generate_session_key()
        client_socket.send(session_key)  # Send session key to the client
        
        print("Session key exchange completed. Awaiting encrypted data...")

        while True:
            encrypted_record = client_socket.recv(1024)  # Receive encrypted data from the client
            if not encrypted_record:
                break  # No more data
            
            decrypted_data = decrypt_data(encrypted_record, session_key)
            print(f"Decrypted Data: {decrypted_data.decode()}")
            # Process the decrypted data
            # ...

    except Exception as e:
        print(f"Error: {e}")
    finally:
        client_socket.close()

def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('0.0.0.0', 9999))  # Bind to all interfaces on port 9999
    server_socket.listen(5)
    print("Server listening on port 9999...")
    
    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Connection from {client_address}")
        handle_client_connection(client_socket)

if __name__ == "__main__":
    start_server()


Server listening on port 9999...
Connection from ('127.0.0.1', 46500)
Handshake initiated.
Received from client: 
Session key exchange completed. Awaiting encrypted data...
Connection from ('127.0.0.1', 57228)
Handshake initiated.
Received from client: 
Session key exchange completed. Awaiting encrypted data...


In [ ]:
import socket
import os
import struct
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
import time
import random
from sympy import nextprime

AES_KEY_SIZE = 32  # AES-256
AES_NONCE_SIZE = 12  # Standard nonce size for AES-GCM
RANDOM_PADDING_MAX = 256  # Max size of random padding
FIXED_PADDING_SIZE = 16  # Fixed padding size for regular TLS
SESSION_KEY_SIZE = 32  # Size for session keys

# Diffie-Hellman key exchange helper functions
def generate_large_prime(bits=2048):
    random_num = os.urandom(bits // 8)
    return nextprime(int.from_bytes(random_num, 'big'))

def key_exchange(private_key, prime_p, generator_g):
    public_key = pow(generator_g, private_key, prime_p)
    return public_key

def derive_session_key(private_key, other_public_key, prime_p):
    shared_secret = pow(other_public_key, private_key, prime_p)
    session_key = SHA256.new(str(shared_secret).encode()).digest()[:SESSION_KEY_SIZE]
    return session_key

def add_random_padding(data):
    padding_length = random.randint(0, RANDOM_PADDING_MAX)
    padding = get_random_bytes(padding_length)
    return data + padding, padding_length

def encrypt_data(plaintext, key):
    nonce = get_random_bytes(AES_NONCE_SIZE)
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data, padding_length = add_random_padding(plaintext)
    ciphertext, tag = cipher.encrypt_and_digest(padded_data)
    return nonce + struct.pack("B", padding_length) + ciphertext + tag

def decrypt_data(record, key):
    nonce = record[:AES_NONCE_SIZE]
    padding_length = struct.unpack("B", record[AES_NONCE_SIZE:AES_NONCE_SIZE + 1])[0]
    tag_start = -16
    ciphertext = record[AES_NONCE_SIZE + 1:tag_start]
    tag = record[tag_start:]

    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    padded_data = cipher.decrypt_and_verify(ciphertext, tag)
    
    return padded_data[:-padding_length] if padding_length > 0 else padded_data

def handle_client_connection(client_socket):
    try:
        print("Handshake initiated.")
        
        # Generate prime and generator for Diffie-Hellman
        prime_p = generate_large_prime()
        generator_g = 2  # Typically, a small number like 2 is used as a generator
        
        # Generate secure random private keys for Alice (server) and Bob (client)
        alice_private = int.from_bytes(os.urandom(32), 'big') % (prime_p - 2) + 1

        # Send the prime and generator to the client
        client_socket.send(f"Prime: {prime_p}\nGenerator: {generator_g}".encode())
        
        # Receive the client's public key
        client_public_key = int(client_socket.recv(1024).decode().strip())

        # Perform the Diffie-Hellman key exchange
        alice_public_key = key_exchange(alice_private, prime_p, generator_g)
        client_socket.send(str(alice_public_key).encode())  # Send server's public key

        # Derive session key for secure communication
        session_key = derive_session_key(alice_private, client_public_key, prime_p)

        print("Session key exchange completed. Awaiting encrypted data...")

        while True:
            encrypted_record = client_socket.recv(1024)  # Receive encrypted data from the client
            if not encrypted_record:
                break  # No more data
            
            decrypted_data = decrypt_data(encrypted_record, session_key)
            print((encrypted_record, session_key))
            print(f"Decrypted Data: {decrypted_data.decode()}")
            response = f"Received your message: {decrypted_data.decode()}"
            encrypted_response = encrypt_data(response.encode(), session_key)
            client_socket.send(encrypted_response)

    except Exception as e:
        print(f"Error: {e}")
    finally:
        client_socket.close()

def start_server():
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.bind(('0.0.0.0', 9999))  # Bind to all interfaces on port 9999
    server_socket.listen(5)
    print("Server listening on port 9999...")
    
    while True:
        client_socket, client_address = server_socket.accept()
        print(f"Connection from {client_address}")
        handle_client_connection(client_socket)

if __name__ == "__main__":
    start_server()


Server listening on port 9999...
Connection from ('127.0.0.1', 42568)
Handshake initiated.
Session key exchange completed. Awaiting encrypted data...
(b"\xd9\xe6\xf1q\xd2\x00\xabGA'\xf9\x82k\xf7\x9f2J\x9d{\x0c\xf3\x9d\xb0\x99T+]\xa9\xb3\x08\xdf\xe7\x85\xc0iG\xfe=\x10\xcd\x96\xcc\xb9'\xb5\x85g\x0b\xf0\x8d\xfa\xc6\x92\x9a%Z\x877{\xf6\xb1\x1b\xeb\x0b\x18\x7f\x0b\xf8\x96MBt4\xc4]7\xf8T\xda\x0f\x8bjo~9\x9bv:\xd5\xfcm\xd2.\x9c>\xc3\x82\xea\xd2Kzn\xa7pO\x83\xcb\xbe3\xf1m\xe8W\x1a\xf7\x07X-\xd4\x7f\xaa\xdfKV\xf7+\x8f\xd7wj\xb0;\x88\x96\x8b!\xa0\x1e\xd7\xc8", b'\xc1\xe7\xe0m\xd6\x0b\x9d\x85F\xf0\x06\x08j\xb6\xa7u\xb0\x12\xd3\x87\xbe\x90\xa5\xdf\x1f\x1a\xed\x86\xb3i\x8c\xab')
Decrypted Data: hiii
(b"\xd6T0+\xae\xfb\xe5\x9a\xe8c\x87\xd4<(Jg\x15\x86\xe6\x99U\xa8\xe2#\x07\x1ek\xbd\xba\xf0 '\xdc>J\xad\x05\xa6\xd9\xa3\xb6Lj7\xe3\x8f\xc8LX\xd6w\x81\xa3\xda\x15,\xb1\x8a\xcb/b\xf9Yv\xe0\xc2\x1e\xfc.\xa3\xc3\x02\x95\x00\xaf\xc8\xbb\xd3V\x8d\xcf\x0b\x8f\x97\xe2&\xadO\xe81\xbfs\xee\t\xe5\xf3\x071n`b\xa1z\x